In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio
from statsmodels.formula.api import ols

# Read in AURN Soton PM2.5 data

In [2]:
AURN_Soton = pd.read_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Southampton\AURN_Soton.csv')

In [3]:
AURN_Soton[:5]

,timestamp,PM2.5_Soton_Centre,Status,Had24
0,2008-11-06 01:00:00,26.0,V ugm-3 (TEOM FDMS),False
1,2008-11-06 02:00:00,18.0,V ugm-3 (TEOM FDMS),False
2,2008-11-06 03:00:00,19.0,V ugm-3 (TEOM FDMS),False
3,2008-11-06 04:00:00,22.0,V ugm-3 (TEOM FDMS),False
4,2008-11-06 05:00:00,25.0,V ugm-3 (TEOM FDMS),False


In [4]:
AURN_Soton['month'] = AURN_Soton.timestamp.str[5:7]

In [5]:
AURN_Soton['year'] = AURN_Soton.timestamp.str[:4]

In [6]:
AURN_Soton = AURN_Soton.rename(columns={'PM2.5_Soton_Centre': 'PM25'})

In [7]:
AURN_Soton = AURN_Soton[['timestamp', 'PM25', 'month', 'year']].dropna()

In [8]:
AURN_Soton_mth = AURN_Soton.groupby(['year','month']).mean()

In [9]:
AURN_Soton_mth[:5]

PM25
year month           
2008 11     10.478114
     12     19.876178
2009 01     22.959072
     02     16.287692
     03     14.958092

In [22]:
AURN_Soton_mth.to_csv(r'D:\Annies_Dissertation\Analysis\Regression\Validation\AURN\AURN_Soton_mth')

In [10]:
AURN_Soton_mth = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\Validation\AURN\AURN_Soton_mth')

# Read in LSOA HES data

In [11]:
Circulatory_mth = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\HES\AllCirculatory_monthly_all.csv')

In [12]:
Circulatory_mth[:5]

,Unnamed: 0,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
0,0,All Circulatory,2009,1,E01015272,NaN,69.247604,8.247776,250.557617,11.430000,22492,7
1,1,All Circulatory,2009,1,E01015273,NaN,147.841522,3.743035,823.720154,33.619999,6665,3
2,2,All Circulatory,2009,1,E01015274,NaN,187.446503,38.159660,549.000244,20.290001,13991,5
3,3,All Circulatory,2009,1,E01015275,NaN,309.000488,59.781487,912.577209,19.780001,14447,5
4,4,All Circulatory,2009,1,E01015276,NaN,162.742996,15.881656,599.259460,26.400000,10063,4


In [13]:
Circulatory_mth = Circulatory_mth[['Disease', 'year', 'month', 'lsoa11', 'n', 'DSR','lcl','ucl','score','rank','decile']].dropna()

In [14]:
Soton_Circ = Circulatory_mth.loc[Circulatory_mth['lsoa11'] == 'E01017153']

In [15]:
Soton_Circ

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
7112,All Circulatory,2009,6,E01017153,7.0,1610.695068,558.276733,3477.020264,43.07,3635,2
16473,All Circulatory,2010,1,E01017153,5.0,799.468689,141.352127,2099.193848,43.07,3635,2


In [16]:
Respiratory_mth = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\HES\AllRespiratory_monthly_all.csv')

In [19]:
Respiratory_mth[:5]

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
2,All Respiratory,2009,1,E01015276,8.0,519.371643,196.146973,1071.320923,26.400000,10063,4
3,All Respiratory,2009,1,E01015277,6.0,607.023254,197.484589,1368.225708,35.400002,5995,2
4,All Respiratory,2009,1,E01015279,5.0,469.725189,136.399750,1128.015015,45.439999,3096,1
9,All Respiratory,2009,1,E01015285,5.0,239.129227,46.085987,620.373474,29.600000,8418,3
14,All Respiratory,2009,1,E01015290,5.0,176.565018,28.419876,469.139282,44.709999,3246,1


In [18]:
Respiratory_mth = Respiratory_mth[['Disease', 'year', 'month', 'lsoa11', 'n', 'DSR','lcl','ucl','score','rank','decile']].dropna()

In [20]:
Soton_Resp = Respiratory_mth.loc[Respiratory_mth['lsoa11'] == 'E01017153']

In [21]:
Soton_Resp

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
11780,All Respiratory,2009,10,E01017153,8.0,723.704285,114.678276,1763.697632,43.07,3635,2
13123,All Respiratory,2009,11,E01017153,7.0,924.347778,223.987198,2166.344971,43.07,3635,2
30034,All Respiratory,2010,12,E01017153,6.0,844.006897,163.821167,2108.256348,43.07,3635,2
32957,All Respiratory,2011,2,E01017153,5.0,192.479034,60.353626,453.415680,43.07,3635,2
63214,All Respiratory,2013,1,E01017153,5.0,473.175629,71.647049,1266.161011,43.07,3635,2


In [24]:
AURN_HES_Soton_R = pd.merge(Soton_Resp, AURN_Soton_mth, on=['year', 'month'], how='inner')

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000,-0.862968,-0.847946,-0.649490,-0.698289,-0.584703,NaN,NaN,NaN,0.621440
month,-0.862968,1.000000,0.757773,0.898944,0.840360,0.874393,NaN,NaN,NaN,-0.343661
n,-0.847946,0.757773,1.000000,0.689452,0.549416,0.653616,NaN,NaN,NaN,-0.688431
DSR,-0.649490,0.898944,0.689452,1.000000,0.901336,0.994907,NaN,NaN,NaN,-0.394112
lcl,-0.698289,0.840360,0.549416,0.901336,1.000000,0.863094,NaN,NaN,NaN,-0.521431
ucl,-0.584703,0.874393,0.653616,0.994907,0.863094,1.000000,NaN,NaN,NaN,-0.319780
score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
decile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM25,0.621440,-0.343661,-0.688431,-0.394112,-0.521431,-0.319780,NaN,NaN,NaN,1.000000


In [41]:
model = ols("DSR ~ PM25", data=AURN_HES_Soton_R)
Respiratory_results = model.fit()
Respiratory_results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                 -0.126
Method:                 Least Squares   F-statistic:                    0.5517
Date:                Tue, 23 Aug 2016   Prob (F-statistic):              0.512
Time:                        20:22:07   Log-Likelihood:                -34.614
No. Observations:                   5   AIC:                             73.23
Df Residuals:                       3   BIC:                             72.45
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1117.4975    669.472      1.669      0.194     -1013.061  3248.056
PM25         -34.0683     45.869     -0.743      0.512      -180.043   111.906
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.341
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.220
Skew:                          -0.310   Prob(JB):                        0.896
Kurtosis:                       2.180   Cond. No.                         69.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## HES MSOA data

In [13]:
Circ_MSOA = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\MSOA\Circ_MSOA.csv')

In [17]:
Circ_MSOA[:5]

,Disease,year,month,MSOA11CD,n,DSR,lcl,ucl,score,rank,decile
8,All Circulatory,2009,1,E02003190,5.0,887.141540,75.347672,2490.366455,71.160004,226,1
16,All Circulatory,2009,1,E02003192,7.0,216.578567,82.616661,454.142334,17.190001,16593,6
22,All Circulatory,2009,1,E02003191,6.0,191.004242,64.210968,426.672730,10.750000,23309,8
24,All Circulatory,2009,1,E02003191,5.0,257.656281,81.810043,604.937927,6.780000,28266,9
43,All Circulatory,2009,1,E02003177,10.0,771.911560,344.309540,1461.249023,8.580000,26015,8


In [16]:
Circ_MSOA = Circ_MSOA[['Disease', 'year', 'month', 'MSOA11CD', 'n', 'DSR','lcl','ucl','score','rank','decile']].dropna()

In [18]:
Soton_Circ = Circ_MSOA.loc[Circ_MSOA['MSOA11CD'] == 'E02003570']

In [19]:
Soton_Circ

,Disease,year,month,MSOA11CD,n,DSR,lcl,ucl,score,rank,decile
7112,All Circulatory,2009,6,E02003570,7.0,1610.695068,558.276733,3477.020264,43.070000,3635,2
16473,All Circulatory,2010,1,E02003570,5.0,799.468689,141.352127,2099.193848,43.070000,3635,2
39363,All Circulatory,2011,6,E02003570,5.0,730.441772,180.043182,1817.432983,58.910000,1026,1
46169,All Circulatory,2011,11,E02003570,5.0,219.117279,66.407608,520.706116,58.910000,1026,1
47515,All Circulatory,2011,12,E02003570,5.0,889.914368,183.669784,2284.688477,38.810001,4858,2
51652,All Circulatory,2012,3,E02003570,5.0,916.797241,121.977104,2486.500488,38.810001,4858,2
67833,All Circulatory,2013,3,E02003570,5.0,620.986450,141.826584,1567.287109,38.810001,4858,2
82824,All Circulatory,2014,2,E02003570,6.0,1171.718628,152.596863,3065.942871,38.810001,4858,2


In [20]:
MSOA_Soton_C = pd.merge(Soton_Circ, AURN_Soton_mth, on=['year', 'month'], how='inner')

In [21]:
MSOA_Soton_C.corr()

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000,-0.315721,-0.255329,-0.210775,-0.573127,-0.071357,-0.313836,0.398737,0.144841,0.056996
month,-0.315721,1.000000,-0.116913,-0.290107,0.022568,-0.379135,0.407395,-0.376525,-0.450988,-0.333633
n,-0.255329,-0.116913,1.000000,0.843970,0.870935,0.741289,-0.211657,0.154427,0.311086,-0.408191
DSR,-0.210775,-0.290107,0.843970,1.000000,0.823512,0.976792,-0.518608,0.464852,0.601052,-0.386775
lcl,-0.573127,0.022568,0.870935,0.823512,1.000000,0.685128,-0.157503,0.086494,0.284470,-0.335161
ucl,-0.071357,-0.379135,0.741289,0.976792,0.685128,1.000000,-0.614121,0.571220,0.673064,-0.363632
score,-0.313836,0.407395,-0.211657,-0.518608,-0.157503,-0.614121,1.000000,-0.993621,-0.977657,-0.092156
rank,0.398737,-0.376525,0.154427,0.464852,0.086494,0.571220,-0.993621,1.000000,0.947715,0.095087
decile,0.144841,-0.450988,0.311086,0.601052,0.284470,0.673064,-0.977657,0.947715,1.000000,0.083538
PM25,0.056996,-0.333633,-0.408191,-0.386775,-0.335161,-0.363632,-0.092156,0.095087,0.083538,1.000000


In [22]:
model = ols("DSR ~ PM25", data=MSOA_Soton_C)
Circ_results = model.fit()
Circ_results.summary()

D:\Anaconda3\lib\site-packages\scipy\stats\stats.py:1557: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.150
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.055
Date:                Wed, 24 Aug 2016   Prob (F-statistic):              0.344
Time:                        18:52:33   Log-Likelihood:                -58.215
No. Observations:                   8   AIC:                             120.4
Df Residuals:                       6   BIC:                             120.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept   1237.2772    385.092      3.213      0.018       294.992  2179.562
PM25         -22.4617     21.864     -1.027      0.344       -75.960    31.037
==============================================================================
Omnibus:                        0.657   Durbin-Watson:                   1.245
Prob(Omnibus):                  0.720   Jarque-Bera (JB):                0.151
Skew:                           0.295   Prob(JB):                        0.927
Kurtosis:                       2.675   Cond. No.                         47.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
Resp_MSOA = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\MSOA\Resp_MSOA.csv')

In [25]:
Resp_MSOA[:5]

,Unnamed: 0,Disease,year,month,n,DSR,lcl,ucl,score,rank,decile,lsoa11,LSOA11NM,MSOA11CD,MSOA11NM
0,0,All Respiratory,2009,1,NaN,78.530869,1.988227,437.546020,20.290001,13991,5,E01015274,Bournemouth 018A,E02003189,Bournemouth 018
1,1,All Respiratory,2009,1,NaN,198.915909,20.564419,729.030701,19.780001,14447,5,E01015275,Bournemouth 016C,E02003187,Bournemouth 016
2,2,All Respiratory,2009,1,8.0,519.371643,196.146973,1071.320923,26.400000,10063,4,E01015276,Bournemouth 016D,E02003187,Bournemouth 016
3,3,All Respiratory,2009,1,6.0,607.023254,197.484589,1368.225708,35.400002,5995,2,E01015277,Bournemouth 016E,E02003187,Bournemouth 016
4,4,All Respiratory,2009,1,5.0,469.725189,136.399750,1128.015015,45.439999,3096,1,E01015279,Bournemouth 019B,E02003190,Bournemouth 019


In [26]:
Resp_MSOA = Resp_MSOA[['Disease', 'year', 'month', 'MSOA11CD', 'n', 'DSR','lcl','ucl','score','rank','decile']].dropna()

In [27]:
Soton_Resp = Resp_MSOA.loc[Resp_MSOA['MSOA11CD'] == 'E02003570']

In [28]:
MSOA_Soton_R = pd.merge(Soton_Resp, AURN_Soton_mth, on=['year', 'month'], how='inner')

In [32]:
MSOA_Soton_R[:5]

,Disease,year,month,MSOA11CD,n,DSR,lcl,ucl,score,rank,decile,PM25
0,All Respiratory,2009,1,E02003570,7.0,507.926605,18.095709,1376.577148,58.910000,1026,1,22.959072
1,All Respiratory,2009,1,E02003570,5.0,1592.745239,306.784241,4132.410645,41.400002,4094,2,22.959072
2,All Respiratory,2009,1,E02003570,5.0,242.721787,68.128426,587.529053,50.590000,2121,1,22.959072
3,All Respiratory,2009,2,E02003570,5.0,520.315674,40.616592,1467.681274,58.910000,1026,1,16.287692
4,All Respiratory,2009,3,E02003570,8.0,1233.148438,267.073944,2882.849854,58.910000,1026,1,14.958092


In [29]:
MSOA_Soton_R.corr()

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000,-0.025892,-0.184855,-0.179332,-0.218976,-0.141070,-0.161314,0.164814,0.187048,0.005504
month,-0.025892,1.000000,0.194094,-0.252768,-0.135505,-0.272868,-0.004268,0.028185,0.045955,-0.407593
n,-0.184855,0.194094,1.000000,0.182049,0.424098,0.060508,0.203192,-0.188389,-0.135449,-0.118210
DSR,-0.179332,-0.252768,0.182049,1.000000,0.708624,0.981139,-0.293878,0.272328,0.296103,0.035781
lcl,-0.218976,-0.135505,0.424098,0.708624,1.000000,0.563878,-0.146386,0.127905,0.140690,-0.035978
ucl,-0.141070,-0.272868,0.060508,0.981139,0.563878,1.000000,-0.319842,0.298433,0.318812,0.057324
score,-0.161314,-0.004268,0.203192,-0.293878,-0.146386,-0.319842,1.000000,-0.994084,-0.950845,0.129787
rank,0.164814,0.028185,-0.188389,0.272328,0.127905,0.298433,-0.994084,1.000000,0.958202,-0.150951
decile,0.187048,0.045955,-0.135449,0.296103,0.140690,0.318812,-0.950845,0.958202,1.000000,-0.133644
PM25,0.005504,-0.407593,-0.118210,0.035781,-0.035978,0.057324,0.129787,-0.150951,-0.133644,1.000000


In [30]:
model = ols("DSR ~ PM25", data=MSOA_Soton_R)
Resp_results = model.fit()
Resp_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                   0.06922
Date:                Wed, 24 Aug 2016   Prob (F-statistic):              0.793
Time:                        18:55:13   Log-Likelihood:                -422.40
No. Observations:                  56   AIC:                             848.8
Df Residuals:                      54   BIC:                             852.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    662.3424    173.680      3.814      0.000       314.134  1010.551
PM25           2.7013     10.267      0.263      0.793       -17.883    23.285
==============================================================================
Omnibus:                       12.221   Durbin-Watson:                   2.190
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               12.669
Skew:                           1.112   Prob(JB):                      0.00177
Kurtosis:                       3.697   Cond. No.                         47.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Bournemouth

In [33]:
AURN_Bmth = pd.read_csv(r'D:\Annies_Dissertation\Methodology\Validation\AURN\Bournemouth\Bournemouth2008_2014.csv')

In [34]:
AURN_Bmth = AURN_Bmth.rename(columns={'PM2.5 particulate matter (Daily measured)':'PM25'})

In [35]:
AURN_Bmth[:5]

,Date,PM25,Status
0,12/03/2009,0.0,V ugm-3 (GRAV EMFAB)
1,30/11/2009,0.0,V ugm-3 (GRAV EMFAB)
2,25/12/2013,0.0,V ugm-3 (GRAV EMFAB)
3,27/12/2009,1.0,V ugm-3 (GRAV EMFAB)
4,16/02/2010,1.0,V ugm-3 (GRAV EMFAB)


In [36]:
AURN_Bmth['month'] = AURN_Bmth.Date.str[3:5]

In [37]:
AURN_Bmth['year'] = AURN_Bmth.Date.str[6:]

In [38]:
AURN_Bmth_mth = AURN_Bmth.groupby(['year','month']).mean()

In [39]:
AURN_Bmth_mth[:5]

PM25
year month           
2009 01     21.354839
     02     13.884615
     03     11.193548
     04     14.000000
     05      5.964286

In [40]:
AURN_Bmth_mth.to_csv(r'D:\Annies_Dissertation\Analysis\Regression\Validation\AURN\AURN_Bmth_mth')

In [41]:
AURN_Bmth_mth = pd.read_csv(r'D:\Annies_Dissertation\Analysis\Regression\Validation\AURN\AURN_Bmth_mth')

## LSOA

In [48]:
Bmth_Circ = Circulatory_mth.loc[Circulatory_mth['lsoa11'] == 'E01015315']

In [62]:
Bmth_Circ[:2]

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
5509,All Circulatory,2009,5,E01015315,5.0,270.068390,82.931976,639.647156,3.95,31334,10
9594,All Circulatory,2009,8,E01015315,9.0,528.581482,231.195541,1020.800598,3.95,31334,10


In [67]:
AURN_HES_Bmth = pd.merge(Bmth_Circ, AURN_Bmth_mth, on=['year', 'month'], how='inner')

In [70]:
AURN_HES_Bmth[:5]

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile,PM25
0,All Circulatory,2009,5,E01015315,5.0,270.068390,82.931976,639.647156,3.95,31334,10,5.964286
1,All Circulatory,2009,8,E01015315,9.0,528.581482,231.195541,1020.800598,3.95,31334,10,4.433333
2,All Circulatory,2010,4,E01015315,5.0,230.563004,74.709770,538.360107,3.95,31334,10,5.500000
3,All Circulatory,2010,12,E01015315,6.0,337.809937,121.351570,740.137390,3.95,31334,10,14.535714
4,All Circulatory,2011,1,E01015315,7.0,392.443512,144.611053,831.941711,3.95,31334,10,7.368421


In [69]:
AURN_HES_Bmth.corr()

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000,-0.342087,-0.306666,-0.316394,-0.315187,-0.313453,NaN,NaN,NaN,0.148944
month,-0.342087,1.000000,0.015763,0.094473,0.082277,0.116981,NaN,NaN,NaN,-0.226039
n,-0.306666,0.015763,1.000000,0.977751,0.988894,0.952001,NaN,NaN,NaN,0.290746
DSR,-0.316394,0.094473,0.977751,1.000000,0.994511,0.994241,NaN,NaN,NaN,0.235374
lcl,-0.315187,0.082277,0.988894,0.994511,1.000000,0.978024,NaN,NaN,NaN,0.252596
ucl,-0.313453,0.116981,0.952001,0.994241,0.978024,1.000000,NaN,NaN,NaN,0.214269
score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
decile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM25,0.148944,-0.226039,0.290746,0.235374,0.252596,0.214269,NaN,NaN,NaN,1.000000


In [73]:
model = ols("DSR ~ PM25", data=AURN_HES_Bmth)
Circulatory_results = model.fit()
Circulatory_results.summary()

D:\Anaconda3\lib\site-packages\scipy\stats\stats.py:1557: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                 -0.017
Method:                 Least Squares   F-statistic:                    0.7625
Date:                Tue, 23 Aug 2016   Prob (F-statistic):              0.398
Time:                        18:39:23   Log-Likelihood:                -85.966
No. Observations:                  15   AIC:                             175.9
Df Residuals:                      13   BIC:                             177.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    292.8880     41.077      7.130      0.000       204.147   381.629
PM25           3.7486      4.293      0.873      0.398        -5.526    13.023
==============================================================================
Omnibus:                       13.141   Durbin-Watson:                   2.685
Prob(Omnibus):                  0.001   Jarque-Bera (JB):                9.366
Skew:                           1.572   Prob(JB):                      0.00925
Kurtosis:                       5.260   Cond. No.                         19.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
Bmth_Resp = Respiratory_mth.loc[Respiratory_mth['lsoa11'] == 'E01015315']

In [36]:
Bmth_Resp

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile
12882,All Respiratory,2009,11,E01015315,5.0,288.887238,93.788933,674.190063,3.95,31334,10
14248,All Respiratory,2009,12,E01015315,5.0,289.138550,88.406960,685.566650,3.95,31334,10
15704,All Respiratory,2010,1,E01015315,6.0,355.459992,117.713852,797.354675,3.95,31334,10
45539,All Respiratory,2011,12,E01015315,5.0,249.314926,78.195862,587.260742,3.95,31334,10
83179,All Respiratory,2014,4,E01015315,5.0,274.900513,83.277054,653.340820,3.95,31334,10


In [37]:
AURN_HES_Bmth_R = pd.merge(Bmth_Resp, AURN_Bmth_mth, on=['year', 'month'], how='inner')

In [38]:
AURN_HES_Bmth_R

,Disease,year,month,lsoa11,n,DSR,lcl,ucl,score,rank,decile,PM25
0,All Respiratory,2009,11,E01015315,5.0,288.887238,93.788933,674.190063,3.95,31334,10,6.400000
1,All Respiratory,2009,12,E01015315,5.0,289.138550,88.406960,685.566650,3.95,31334,10,7.833333
2,All Respiratory,2010,1,E01015315,6.0,355.459992,117.713852,797.354675,3.95,31334,10,14.419355
3,All Respiratory,2011,12,E01015315,5.0,249.314926,78.195862,587.260742,3.95,31334,10,4.903226
4,All Respiratory,2014,4,E01015315,5.0,274.900513,83.277054,653.340820,3.95,31334,10,7.716897


In [39]:
AURN_HES_Bmth_R.corr()

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000e+00,-0.444976,-1.617492e-01,-3.187163e-01,-3.745942e-01,-3.134202e-01,-4.903667e-14,NaN,NaN,-1.067109e-01
month,-4.449759e-01,1.000000,-7.601514e-01,-7.022010e-01,-6.618880e-01,-6.900982e-01,0.000000e+00,NaN,NaN,-8.215030e-01
n,-1.617492e-01,-0.760151,1.000000e+00,9.106598e-01,9.258452e-01,8.657581e-01,4.440892e-16,NaN,NaN,9.454844e-01
DSR,-3.187163e-01,-0.702201,9.106598e-01,1.000000e+00,9.854624e-01,9.935910e-01,-4.859063e-16,NaN,NaN,9.621843e-01
lcl,-3.745942e-01,-0.661888,9.258452e-01,9.854624e-01,1.000000e+00,9.630197e-01,2.068937e-16,NaN,NaN,9.273933e-01
ucl,-3.134202e-01,-0.690098,8.657581e-01,9.935910e-01,9.630197e-01,1.000000e+00,-1.670886e-15,NaN,NaN,9.530606e-01
score,-4.903667e-14,0.000000,4.440892e-16,-4.859063e-16,2.068937e-16,-1.670886e-15,1.000000e+00,NaN,NaN,2.724370e-16
rank,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
decile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM25,-1.067109e-01,-0.821503,9.454844e-01,9.621843e-01,9.273933e-01,9.530606e-01,2.724370e-16,NaN,NaN,1.000000e+00


In [40]:
model = ols("DSR ~ PM25", data=AURN_HES_Bmth_R)
Circulatory_results = model.fit()
Circulatory_results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\stats\stattools.py:72: UserWarning: omni_normtest is not valid with less than 8 observations; 5 samples were given.
  "samples were given." % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     37.43
Date:                Tue, 23 Aug 2016   Prob (F-statistic):            0.00878
Time:                        20:20:59   Log-Likelihood:                -18.383
No. Observations:                   5   AIC:                             40.77
Df Residuals:                       3   BIC:                             39.98
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    206.0400     15.026     13.713      0.001       158.222   253.858
PM25          10.3579      1.693      6.118      0.009         4.970    15.746
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.627
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.456
Skew:                           0.633   Prob(JB):                        0.796
Kurtosis:                       2.235   Cond. No.                         24.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## MSOA

In [42]:
Bmth_Circ = Circ_MSOA.loc[Circ_MSOA['MSOA11CD'] == 'E02003182']

In [43]:
MSOA_Bmth_C = pd.merge(Bmth_Circ, AURN_Bmth_mth, on=['year', 'month'], how='inner')

In [50]:
MSOA_Bmth_C

,Disease,year,month,MSOA11CD,n,DSR,lcl,ucl,score,rank,decile,PM25
0,All Circulatory,2009,4,E02003182,7.0,582.803101,233.432037,1202.366577,14.69,18956,6,14.000000
1,All Circulatory,2009,4,E02003182,5.0,347.073059,103.863800,827.390991,9.48,24840,8,14.000000
2,All Circulatory,2009,5,E02003182,5.0,386.488464,122.996841,906.862427,10.57,23527,8,5.964286
3,All Circulatory,2009,5,E02003182,5.0,270.068390,82.931976,639.647156,3.95,31334,10,5.964286
4,All Circulatory,2009,7,E02003182,5.0,308.963135,97.740341,726.110474,10.57,23527,8,5.296296
5,All Circulatory,2009,7,E02003182,5.0,347.452484,112.674393,811.119568,14.69,18956,6,5.296296
6,All Circulatory,2009,8,E02003182,9.0,528.581482,231.195541,1020.800598,3.95,31334,10,4.433333
7,All Circulatory,2009,9,E02003182,6.0,338.632416,117.967583,748.778381,10.57,23527,8,5.933333
8,All Circulatory,2009,11,E02003182,6.0,376.218506,132.310608,829.565918,10.57,23527,8,6.400000
9,All Circulatory,2009,11,E02003182,5.0,415.386108,133.885498,971.326294,14.69,18956,6,6.400000


In [44]:
MSOA_Bmth_C.corr()

,year,month,n,DSR,lcl,ucl,score,rank,decile,PM25
year,1.000000,-0.198217,-0.011996,0.028763,0.013106,0.038607,-0.188332,0.188920,0.171609,0.130983
month,-0.198217,1.000000,-0.035850,-0.034633,-0.021109,-0.040477,0.037182,-0.038875,-0.017822,-0.349622
n,-0.011996,-0.035850,1.000000,0.780869,0.889189,0.642315,-0.036460,0.036506,0.023512,0.166598
DSR,0.028763,-0.034633,0.780869,1.000000,0.974474,0.978492,0.402482,-0.403838,-0.423236,0.084565
lcl,0.013106,-0.021109,0.889189,0.974474,1.000000,0.907671,0.290498,-0.291010,-0.310304,0.107303
ucl,0.038607,-0.040477,0.642315,0.978492,0.907671,1.000000,0.498836,-0.500733,-0.519860,0.060835
score,-0.188332,0.037182,-0.036460,0.402482,0.290498,0.498836,1.000000,-0.999877,-0.992794,0.072161
rank,0.188920,-0.038875,0.036506,-0.403838,-0.291010,-0.500733,-0.999877,1.000000,0.991125,-0.073393
decile,0.171609,-0.017822,0.023512,-0.423236,-0.310304,-0.519860,-0.992794,0.991125,1.000000,-0.060159
PM25,0.130983,-0.349622,0.166598,0.084565,0.107303,0.060835,0.072161,-0.073393,-0.060159,1.000000


In [45]:
model = ols("DSR ~ PM25", data=MSOA_Bmth_C)
Circulatory_results = model.fit()
Circulatory_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.3097
Date:                Wed, 24 Aug 2016   Prob (F-statistic):              0.581
Time:                        19:00:47   Log-Likelihood:                -282.38
No. Observations:                  45   AIC:                             568.8
Df Residuals:                      43   BIC:                             572.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    390.5991     43.423      8.995      0.000       303.027   478.171
PM25           2.4456      4.394      0.557      0.581        -6.417    11.308
==============================================================================
Omnibus:                       10.337   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.006   Jarque-Bera (JB):                9.907
Skew:                           1.097   Prob(JB):                      0.00706
Kurtosis:                       3.687   Cond. No.                         22.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
Bmth_Resp = Resp_MSOA.loc[Resp_MSOA['MSOA11CD'] == 'E02003182']

In [47]:
MSOA_Bmth_R = pd.merge(Bmth_Resp, AURN_Bmth_mth, on=['year', 'month'], how='inner')

In [51]:
MSOA_Bmth_R

,Disease,year,month,MSOA11CD,n,DSR,lcl,ucl,score,rank,decile,PM25
0,All Respiratory,2009,11,E02003182,5.0,288.887238,93.788933,674.190063,3.95,31334,10,6.400000
1,All Respiratory,2009,12,E02003182,5.0,289.138550,88.406960,685.566650,3.95,31334,10,7.833333
2,All Respiratory,2010,1,E02003182,6.0,355.459992,117.713852,797.354675,3.95,31334,10,14.419355
3,All Respiratory,2010,7,E02003182,5.0,335.133026,88.323647,822.560913,9.48,24840,8,4.428571
4,All Respiratory,2011,11,E02003182,5.0,319.246002,100.983063,750.297058,10.57,23527,8,12.000000
5,All Respiratory,2011,12,E02003182,5.0,249.314926,78.195862,587.260742,3.95,31334,10,4.903226
6,All Respiratory,2013,12,E02003182,5.0,448.958038,136.526871,1065.983521,9.48,24840,8,7.585484
7,All Respiratory,2014,4,E02003182,5.0,274.900513,83.277054,653.340820,3.95,31334,10,7.716897


MSOA_Bmth_R.corr()

In [49]:
model = ols("DSR ~ PM25", data=MSOA_Bmth_R)
Circulatory_results = model.fit()
Circulatory_results.summary()

D:\Anaconda3\lib\site-packages\scipy\stats\stats.py:1557: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    DSR   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                 -0.081
Method:                 Least Squares   F-statistic:                    0.4739
Date:                Wed, 24 Aug 2016   Prob (F-statistic):              0.517
Time:                        19:00:50   Log-Likelihood:                -43.554
No. Observations:                   8   AIC:                             91.11
Df Residuals:                       6   BIC:                             91.27
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    280.0600     62.536      4.478      0.004       127.039   433.081
PM25           4.9100      7.133      0.688      0.517       -12.543    22.363
==============================================================================
Omnibus:                        9.373   Durbin-Watson:                   2.845
Prob(Omnibus):                  0.009   Jarque-Bera (JB):                3.122
Skew:                           1.445   Prob(JB):                        0.210
Kurtosis:                       4.008   Cond. No.                         24.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""